# Final Exam 2021

- The short answers were written down on paper

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import kurtosis, skew
from scipy.stats import norm
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader as dr
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[15, 6]
import matplotlib.cm as cm

## Part 2| Portfolio Analysis



Using the **midterm_2_pricing.xlsx** sheet, what are the weights of the tangecny portfolio formed by all of these commodities

- Use the sheet **assets (excess returns)**

In [2]:
midterm_data_path = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory\midterm_2_data_pricing.xlsx'
assets = pd.read_excel(midterm_data_path,sheet_name='assets (excess returns)').set_index("Date")

In [3]:
assets.head()

,NG1,KC1,CC1,LB1,CT1,SB1,LC1,W1,S1,C1,GC1,SI1,HG1,PA1
Date,,,,,,,,,,,,,,
2000-01-31,0.138880,-0.121654,-0.054279,-0.010977,0.136223,-0.118479,0.020436,0.027087,0.096062,0.071695,-0.026199,-0.027377,-0.014071,0.074929
2000-02-29,0.032890,-0.105110,-0.057130,-0.051617,-0.026077,-0.146366,-0.010002,-0.040398,-0.017587,-0.027027,0.034542,-0.049505,-0.071836,0.464650
2000-03-31,0.061943,0.033338,0.057717,-0.021357,0.022508,0.264117,0.029995,0.057041,0.083579,0.092974,-0.058405,-0.010247,0.011818,-0.168324
2000-04-30,0.061953,-0.085603,-0.070850,-0.082210,-0.046396,-0.130024,0.024221,-0.080863,-0.039430,-0.056507,-0.017890,-0.016552,-0.017100,0.037549
2000-05-31,0.381819,-0.029134,0.122175,-0.013347,0.119394,0.458178,-0.092273,0.129159,-0.022094,0.000587,-0.015921,-0.008831,0.021582,-0.067440


In [4]:
def tangency_portfolio_rfr(asset_return,cov_matrix):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector when a riskless assset is available
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num

In [5]:
# Calculating the tangency portfolio with risk-free asset available
tan_weights = tangency_portfolio_rfr(assets.mean(),assets.cov())
tan_weights = pd.DataFrame(tan_weights, index = assets.columns, columns = ["Tangency Weights"])
tan_weights

,Tangency Weights
NG1,0.057447
KC1,-0.072836
CC1,0.074472
LB1,0.086633
CT1,-0.009538
SB1,0.063606
LC1,0.128941
W1,-0.010437
S1,0.027315
C1,0.084830


 Report the mean, volatility, and Sharpe ratio of the tangency portfolio. Annualize all
three statistics.

In [6]:
def mvo_performance_stats(asset_returns,cov_matrix,port_weights, port_type,period):
    """ 
        Returns the Annualized Performance Stats for given asset returns, portfolio weights and covariance matrix
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            port_weights = weights of the assets in the portfolio (1 x n) Vector
            port_type = Type of Portfolio | Eg - Tangency or Mean-Variance Portfolio
            period = Monthly frequency
    """
    
    ret = np.dot(port_weights,asset_returns)*period
    vol = np.sqrt(port_weights @ cov_matrix @ port_weights.T)*np.sqrt(period)
    sharpe = ret/vol

    stats = pd.DataFrame([[ret,vol,sharpe]],columns= ["Annualized Return","Annualized Volatility","Annualized Sharpe Ratio"], index = [port_type])
    return stats

In [7]:
mvo_performance_stats(assets.mean(), assets.cov(),port_weights = tangency_portfolio_rfr(assets.mean(), assets.cov()), port_type='Tangency', period = 12)

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
Tangency,0.087513,0.116314,0.752389


# 2.3
- Compute the Value-at-Risk at the 5th percentile calculateed from the historical data
- Does the mean divided by the VaR compare favorably to the mean-divided-by_VaR for the individual commodities

In [8]:
# Computing the 5th percent quantile
rets_tan = (assets@tangency_portfolio_rfr(assets.mean(), assets.cov()))
print(f"The 5th quantile of the tangency portfolio return is {np.round(rets_tan.quantile(.05),5)*100}%")

The 5th quantile of the tangency portfolio return is -4.139%


In [9]:
MeanVaR = pd.DataFrame(assets.mean()/assets.quantile(.05), columns = ['Mean/VaR'])
MeanVaR.loc['Tangency'] = rets_tan.mean()/rets_tan.quantile(.05)
MeanVaR

,Mean/VaR
NG1,-0.060425
KC1,-0.030680
CC1,-0.054331
LB1,-0.079256
CT1,-0.036003
SB1,-0.054994
LC1,-0.026468
W1,-0.050229
S1,-0.048536
C1,-0.049556


- Yes, the mean divided by the value at risk compares favorably to the mean divided by value at risk for individual commodites because of the higher expected returns given  by the tangency portfolio. This results in a higher absolute number

Question 4 : Use the data through 2017 to estimate the tangency weights. Apply these tangency
weights to the data from 2018-2021.
Report the mean, volatility, and Sharpe ratio of this “out-of-sample” tangency portfolio. Annualize the statistics.

In [10]:
rets_training = assets.loc[:'2017']
# Calculating the tangency weights
rets_training_tangency = tangency_portfolio_rfr(rets_training.mean(),rets_training.cov())
pd.DataFrame(rets_training_tangency, index = assets.columns,columns = ["Train Tan Weights"])

,Train Tan Weights
NG1,0.051154
KC1,-0.094784
CC1,0.081385
LB1,0.041725
CT1,0.015333
SB1,0.082755
LC1,0.172524
W1,-0.002869
S1,0.063006
C1,0.009942


In [11]:
# Calculating OOS performance of this tangency portfolio

OOS_rets = assets.loc['2018':]@rets_training_tangency
def performance_summary(returns, annual_frac = 12):
    df = pd.DataFrame(data = None, columns = ['Peformance Summary'])
    df.loc['Mean'] = returns.mean()*annual_frac
    df.loc['Volatility'] = returns.std()*np.sqrt(annual_frac)
    df.loc['Sharpe'] = df.loc['Mean']/df.loc['Volatility']
    return df

In [12]:
performance_summary(OOS_rets)

,Peformance Summary
Mean,0.074084
Volatility,0.098937
Sharpe,0.748799


Question 2.5
What aspect of Mean-Variance Optimization makes out-of-sample performance fragile?

**Answer:** A slight change in the correlation and expected returns of the assets can make the inverted covariance matrix very explosive resulting in a completely different portfolio. The issue does not seem pertinent in the implementation above considering the performance statistics are similar. 

Question 2.6| 

- Consider the data for crude oil, (CL1), found in the same file, but on sheet “factors (excess
returns)”. Suppose an investor wants to hold crude oil (CL1), but wants to hedge out the
exposure to NG1 and KC1 (both series are found on the assets (excess returns) tab we’ve
been using.)

(a) (5pts) Report the regression-based hedge ratio. Do NOT include an intercept in the regression.


(b) (5pts) Report the mean, volatility, and Sharpe ratios of CL1 and the hedged version of
CL1. Annualize the statistics.


In [13]:
factor_crude = pd.read_excel(midterm_data_path,sheet_name = 'factors (excess returns)').drop(columns = "MKT").set_index("Date")
factor_crude.head()

,CL1
Date,
2000-01-31,0.075587
2000-02-29,0.096641
2000-03-31,-0.120704
2000-04-30,-0.047723
2000-05-31,0.122040


In [14]:
# Calculating the Hedge Ratios

reg_replication = sm.OLS(factor_crude['CL1'], assets[['NG1','KC1']]).fit()

reg_replication.params.to_frame('Hedge Ratio')


,Hedge Ratio
NG1,0.131032
KC1,0.106365


In [15]:
performance_summary(factor_crude['CL1'])

,Peformance Summary
Mean,0.108693
Volatility,0.391642
Sharpe,0.277532


In [16]:
hedged_returns = (factor_crude['CL1'] - assets[['NG1','KC1']]@reg_replication.params.T)
performance_summary(hedged_returns)

,Peformance Summary
Mean,0.085045
Volatility,0.383948
Sharpe,0.221502


# Section 3| Cases

 Consider the firms we studied in the quarter: DFA, GMO, and LTCM 

- Directional positions
- Factor investing
- Spread trades
- Liquidity provider
- Value investors
- Arbitrage-oriented
- Funding risk
- Adverse selection risk

GMO made investments based on expected future directions of certain asset classes using forecasting. 
DFA utilized factor investing. Booth originally wanted to harvest the small minus big factor permium.
LTCM engaged heavily in spread trades using swaps
LTCM was a major liquidity provider to large financial institutions| DFA is a liquidity provider for small stocks
DFA utilized value investing using the value factor for investors and GMO utilized value driven forecasting
LTCM was heavily arbitrage oriented based on the belief that spreads between interest rates would converge in the long-run no matter what
LTCM had major funding risk in their swap positions. If the bond they were long in defaulted then they would fail to pay off the position they were leveraged in- 12 month repos
DFA had the largest adverse selection risk since the lemons problem is mostly associated with small cap stocks


Question 3.2

Did we find LTCM to have significant SPY exposure, linearly or non-linearly? How about GMO?

LTCM had significant non-linear exposure to SPY- This was confirmed by the quadratic regression.


GMO had a large exposure to the SPY with a beta of around 0.5 

# Section 4| The Expectations Hypothesis

4.1

- If the EH was true, alpha would be 0 and beta would also be 0.
- The r-squared would not have any meaningful interpretation because the EH states that returns are unpredictable

4.1
b
- The table provides strong evidence against the EH because the betas are all closer to 1 than 0. 

4.1
c
- In order to to see if the evidence is against the EH we need to to perform a t-test on the betas to determine whether or not the coffecients are statistically insignificant from 0. Yet, the table shows that the EH is false and forward spreads can predict excess returns on bond

4.2
a
- If the EH was true for the one-period bond yield on n-period forward spread, then we would expect alpha to equal 0 and beta to equal 1. The current forward rates should be the best predictor of bond yields. 
- The r-squared provides no meaninful interpretation

4.2
b
- The results are against the EH since the betas are closer to 0 than they are to 1.

4.2
c
- The evidence is consistently against the EMH in all the rows.

4.3

There can be a myriad of reasons to explain why the EH fails for both regressions. 
- The advice about trading bonds could include:
- Bonds with higher yields are expected to have higher returns, even if not held to maturity.
In times of higher forward rates or steeper yield curve, we expect long-term bonds to have particularly high excess returns.
If the yield curve is flat or inverted---or if the forward curve is flat or inverted--we expect short-term bonds to do relatively better.
Forward rates are not the optimal predictor of future yields, they are a predictor of high excess returns.

# Section 5| Forecasting

- Forecast SPY using three macro signals

- Find the macro signal data in “final exam data.xlsx”, on sheet “signals”.
- Find the SPY returns in “final exam data.xlsx”, on sheet “spy’ (total returns)’.
- As in Homework 7, the data is provided such that any row’s date is reporting values known as
of that date.


In [17]:
def regression_based_performance(factor,fund_ret,rf,constant = True):
    """ 
        Returns the Regression based performance Stats for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            rf - Monthly risk free rate of return
        Output:
            summary_stats - (Beta of regression, treynor ratio, information ratio, alpha). 
            
    """
    if constant:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if constant:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6) *12

        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret - rf).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if constant:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if constant:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error,model.resid)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error,model.resid)

In [18]:
def reg_params(df, y_col, X_col, intercept = True, annual_fac=12):
    y = df[y_col]
    if intercept == True:
        X = sm.add_constant(df[X_col])
    else:
        X = df[X_col]
    
    model = sm.OLS(y, X, missing = 'drop').fit()
    reg_df = model.params.to_frame('Regression Parameters')
    reg_df.loc['Rsquared'] = model.rsquared
    
    if intercept == True:
        reg_df.loc['const'] *= annual_fac
    
    return reg_df

- Consider the lagged regression:
- Regress SPY returns on last month returns. Report the r-squared and the usual OLS estimates

In [19]:
final_exam_2021 = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/final_exam_data_2021.xlsx'
spy_rets = pd.read_excel(final_exam_2021,sheet_name = 'spy (total returns)').set_index("date")
signals = pd.read_excel(final_exam_2021,sheet_name = 'signals').set_index('date')
signals_lag = signals.shift().dropna()

In [20]:
signals_lag

,Level,Slope,Inflation Growth
date,,,
1993-03-31,6.03,2.11,-0.297517
1993-04-30,6.03,2.07,-0.637352
1993-05-31,6.05,2.22,-0.655366
1993-06-30,6.16,1.92,-0.386430
1993-07-31,5.80,1.77,-0.292412
...,...,...,...
2021-06-30,1.58,1.44,0.546527
2021-07-31,1.45,1.20,0.544357
2021-08-31,1.24,1.05,0.007303


In [21]:
spy_rets.head()

,SPY
date,
1993-02-28,0.010667
1993-03-31,0.022408
1993-04-30,-0.025589
1993-05-31,0.026970
1993-06-30,0.003667


In [22]:
spy_signal_df = spy_rets.join(signals_lag, how = 'inner')
spy_signal_df.head()

,SPY,Level,Slope,Inflation Growth
date,,,,
1993-03-31,0.022408,6.03,2.11,-0.297517
1993-04-30,-0.025589,6.03,2.07,-0.637352
1993-05-31,0.026970,6.05,2.22,-0.655366
1993-06-30,0.003667,6.16,1.92,-0.386430
1993-07-31,-0.004855,5.80,1.77,-0.292412


In [23]:
reg_params(spy_signal_df, 'SPY',['Level','Slope','Inflation Growth'])

,Regression Parameters
const,0.285357
Level,-0.002312
Slope,-0.005513
Inflation Growth,-0.012906
Rsquared,0.029638


In [24]:
X = sm.add_constant(spy_signal_df[['Level','Slope','Inflation Growth']])
lagged_regression = sm.OLS(spy_signal_df['SPY'], X).fit()
forecasts = lagged_regression.predict(X)
forecasts.head()

date
1993-03-31    0.002046
1993-04-30    0.006653
1993-05-31    0.006012
1993-06-30    0.003941
1993-07-31    0.004387
dtype: float64

## 5.2
- Use the forecasted SPY returns to build trading weights scaled by 100 and calculate the return on this strategy

- Report the first and last 5 values

In [25]:
wt = 100*forecasts
returns = spy_signal_df['SPY']*wt
trading_strat = pd.DataFrame(returns, columns=["Active"])
trading_strat['Passive Strategy'] = spy_signal_df['SPY']

In [26]:
trading_strat

,Active,Passive Strategy
date,,
1993-03-31,0.004585,0.022408
1993-04-30,-0.017024,-0.025589
1993-05-31,0.016214,0.026970
1993-06-30,0.001445,0.003667
1993-07-31,-0.002130,-0.004855
...,...,...
2021-06-30,0.011539,0.022473
2021-07-31,0.016567,0.024412
2021-08-31,0.044729,0.029760


# 5.3 
- Calculate annualized mean,volatility, sharpe, and max-drawdown from both the active and passive strategy

In [27]:
def performance_summary(return_data):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*12)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(12))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.5)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.5)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [28]:
performance_summary(trading_strat)

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.5),CVaR (0.5),Max Drawdown,Peak,Bottom,Recovery
Active,0.166562,0.177158,0.940189,0.625333,5.602101,-0.064788,-0.101122,-0.283313,2007-10-31,2009-02-28,2009-08-31
Passive Strategy,0.111393,0.146190,0.761975,-0.618233,1.252535,-0.069397,-0.094062,-0.507980,2007-10-31,2009-02-28,2012-03-31


In [29]:
spy_signal_df

,SPY,Level,Slope,Inflation Growth
date,,,,
1993-03-31,0.022408,6.03,2.11,-0.297517
1993-04-30,-0.025589,6.03,2.07,-0.637352
1993-05-31,0.026970,6.05,2.22,-0.655366
1993-06-30,0.003667,6.16,1.92,-0.386430
1993-07-31,-0.004855,5.80,1.77,-0.292412
...,...,...,...,...
2021-06-30,0.022473,1.58,1.44,0.546527
2021-07-31,0.024412,1.45,1.20,0.544357
2021-08-31,0.029760,1.24,1.05,0.007303


## 5.4

- Run a linear factor decomposition of regressing active returns on passive returns and report the market alpha, beta, and information ratio

In [30]:
strat_reg = regression_based_performance(trading_strat['Passive Strategy'], trading_strat['Active'], rf = 0)

alpha = strat_reg[3]
beta = strat_reg[0][0]
information_ratio = strat_reg[2]
pd.DataFrame([alpha, beta,information_ratio], index = ['Alpha','Beta','Information Ratio'], columns = ['Statistics'])

,Statistics
Alpha,0.064992
Beta,0.911759
Information Ratio,0.556972


In [31]:
passive_reg = regression_based_performance(spy_signal_df['SPY'], trading_strat['Passive Strategy'], rf = 0)

alpha_1 = passive_reg[3]
beta_1 = passive_reg[0][0]
info_ratio = passive_reg[2]
pd.DataFrame([alpha_1, beta_1, info_ratio], index = ['Alpha','Beta','Information Ratio'], columns = ['Passive Regression Performance'])

,Passive Regression Performance
Alpha,-0.000000
Beta,1.000000
Information Ratio,-6.174566


## 5.5 and 5.6

- Let's consider the out-of-sample performacne of the strategy.

Forecast values of SPY for January 2000 through Oct 2021. (So we are using the data up
until January 2000 as “burn-in” data.)
- Loop through time, estimating (1) only using data through time t.
- Use the estimated parameters of (1), along with xt+1 to calculate the out-of-sample forecast
for the following period, t + 1.


In [32]:
start_time = pd.to_datetime('1999-12-31')
forecasted_OOS = pd.DataFrame(columns = ["Forecast"], index = spy_signal_df.index, dtype= 'float64')
Y_ = spy_signal_df['SPY']
X = signals[['Level',"Slope","Inflation Growth"]]
X_ = spy_signal_df[['Level',"Slope","Inflation Growth"]]
est = LinearRegression()


In [33]:
for t in spy_signal_df.loc[start_time:,:].index:
    yt = Y_.loc[:t].values.reshape(-1,1)
    Xlag_t = X_.loc[:t,:].values
    x_t = X.loc[t,:].values.reshape(-1,1)
    est.fit(Xlag_t,yt);
    predval = est.predict(x_t.T)[0,0]
    forecasted_OOS.loc[t,'Forecast'] = predval
    

forecasted_OOS = forecasted_OOS.shift().dropna()
# Include an expanded mean (baseline forecast) that uses all SPY Data

    

In [34]:
spy_expanding = spy_rets.expanding().mean().dropna()
spy_expanding = spy_expanding.shift()
forecasted_OOS = forecasted_OOS.join(spy_expanding, how = 'inner')
forecasted_OOS

,Forecast,SPY
date,,
2000-01-31,0.020853,0.016951
2000-02-29,0.025365,0.016156
2000-03-31,0.027119,0.015787
2000-04-30,0.038440,0.016730
2000-05-31,0.030322,0.016134
...,...,...
2021-06-30,0.004365,0.009129
2021-07-31,0.006247,0.009168
2021-08-31,0.014917,0.009212


In [35]:
forecasted_OOS.columns = ['Forecast','Mean']

- **5.6** Build a baseline forecast of SPY which uses the mean of all the preceding SPY data (going all the way back to Feb 1993)

In [36]:
forecasted_OOS

,Forecast,Mean
date,,
2000-01-31,0.020853,0.016951
2000-02-29,0.025365,0.016156
2000-03-31,0.027119,0.015787
2000-04-30,0.038440,0.016730
2000-05-31,0.030322,0.016134
...,...,...
2021-06-30,0.004365,0.009129
2021-07-31,0.006247,0.009168
2021-08-31,0.014917,0.009212


**5.7**

Report the out-of-sample R-squared relative to your baseline forecast

In [37]:
def oos_rsquared(data,forecasts,null=None):
    data = data.copy()
    forecasts = forecasts.copy()
    null = null.copy()
    
    # if no Null forecast given, use expanding mean
    if null is None:
        null = data.expanding().mean().shift()

    # label Data and Null accordingly--input may be series or dataframe
    if isinstance(null, pd.DataFrame):
        null.columns = ['Null']
    elif isinstance(null,pd.Series):
        null.name = 'Null'
    if isinstance(data, pd.DataFrame):
        data.columns = ['Data']
    elif isinstance(data,pd.Series):
        data.name = 'Data'

    # double check data is aligned and no NaN (null will have NaN in first value by default)
    alldata = forecasts.join(data,how='inner',rsuffix='_Data').join(null,how='inner',rsuffix='_Null').dropna(axis=0)
    null = alldata[['Null']]
    data = alldata[['Data']]
    forecasts = alldata.drop(columns=['Data','Null'])


    # Forecast MSE
    err_forecast = forecasts.subtract(data.values)
    mse_forecast = (err_forecast**2).sum()

    # Null MSE
    err_null = null.subtract(data.values)
    mse_null = (err_null**2).sum()

    # OOS R-squared
    r2oos = (1 - mse_forecast/mse_null.values).to_frame().T
    r2oos.index = ['OOS-Rsquared']

    return r2oos


In [38]:
spy_OOS= spy_rets.join(forecasted_OOS, how = 'inner')
spy_OOS.head()

,SPY,Forecast,Mean
date,,,
2000-01-31,-0.049787,0.020853,0.016951
2000-02-29,-0.015226,0.025365,0.016156
2000-03-31,0.096896,0.027119,0.015787
2000-04-30,-0.035121,0.038440,0.016730
2000-05-31,-0.015722,0.030322,0.016134


In [39]:
oos_rsquared(spy_OOS['SPY'], forecasted_OOS, forecasted_OOS['Mean'])

,Forecast,Mean
OOS-Rsquared,-0.010991,0.0


In [40]:
MSE = sum((spy_OOS['Forecast']-spy_OOS['SPY'])**2)
TSS = sum((spy_OOS['Mean']-spy_OOS['SPY'])**2)

OOS = 1-(MSE/TSS)
OOS
# My answers are a litte off from last year's midterm because of rounding errors

-0.010991196168173456

**Report the Correlation of the two forecasts of SPY to the actual values of SPY**


In [41]:
forecasted_OOS.corrwith(spy_OOS['SPY']).to_frame('Correlation')


,Correlation
Forecast,0.072857
Mean,-0.162329


In [42]:
forecasted_OOS.head()

,Forecast,Mean
date,,
2000-01-31,0.020853,0.016951
2000-02-29,0.025365,0.016156
2000-03-31,0.027119,0.015787
2000-04-30,0.038440,0.016730
2000-05-31,0.030322,0.016134


In [43]:
spy_OOS

,SPY,Forecast,Mean
date,,,
2000-01-31,-0.049787,0.020853,0.016951
2000-02-29,-0.015226,0.025365,0.016156
2000-03-31,0.096896,0.027119,0.015787
2000-04-30,-0.035121,0.038440,0.016730
2000-05-31,-0.015722,0.030322,0.016134
...,...,...,...
2021-06-30,0.022473,0.004365,0.009129
2021-07-31,0.024412,0.006247,0.009168
2021-08-31,0.029760,0.014917,0.009212


# 5.9

- Trade on the two forecasts, using the same 100x weighting scheme as above. Report the first and last 5 values 

In [44]:
wts_OOS = 100*forecasted_OOS
fund_returns_OOS = spy_OOS['SPY']*wts_OOS.T

fund_returns_OOS = fund_returns_OOS.T
fund_returns_OOS['Passive'] = spy_OOS['SPY']
fund_returns_OOS

,Forecast,Mean,Passive
date,,,
2000-01-31,-0.103823,-0.084392,-0.049787
2000-02-29,-0.038621,-0.024599,-0.015226
2000-03-31,0.262774,0.152969,0.096896
2000-04-30,-0.135005,-0.058757,-0.035121
2000-05-31,-0.047673,-0.025366,-0.015722
...,...,...,...
2021-06-30,0.009809,0.020514,0.022473
2021-07-31,0.015251,0.022381,0.024412
2021-08-31,0.044394,0.027416,0.029760


**5.10** 
Report the univariate statistics for the two OOS strategies



In [45]:
performance_summary(fund_returns_OOS)

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.5),CVaR (0.5),Max Drawdown,Peak,Bottom,Recovery
Forecast,0.113772,0.214491,0.530427,0.216023,4.101308,-0.094761,-0.133755,-0.505123,2000-03-31,2003-02-28,2009-02-28
Mean,0.055994,0.135737,0.412519,-0.457888,1.730965,-0.073026,-0.091415,-0.517748,2000-08-31,2002-09-30,2013-07-31
Passive,0.082307,0.150449,0.547079,-0.504538,1.053584,-0.074608,-0.095652,-0.507980,2007-10-31,2009-02-28,2012-03-31


In [46]:
# Reporting the Linear Factor Decomposition

def OLS_stats(y,df, annualization = 12):
    dataframe = pd.DataFrame(columns = ['Univariate Stats'])
    X = sm.add_constant(df['Passive'])
    reg = sm.OLS(y,X).fit()
    dataframe.loc['Alpha'] = reg.params[0]*annualization
    dataframe.loc['Beta'] = reg.params[1]
    dataframe.loc['Information Ratio'] = reg.params[0]*annualization/(reg.resid.std()*np.sqrt(annualization))
    
    return dataframe
    
    
    

In [47]:
OLS_stats(fund_returns_OOS['Forecast'], fund_returns_OOS)

,Univariate Stats
Alpha,0.065813
Beta,0.582675
Information Ratio,0.336195


In [48]:
OLS_stats(fund_returns_OOS['Mean'], fund_returns_OOS)

,Univariate Stats
Alpha,-0.015664
Beta,0.870620
Information Ratio,-0.439948


In [49]:
OLS_stats(fund_returns_OOS['Passive'], fund_returns_OOS)

,Univariate Stats
Alpha,2.081668e-17
Beta,1.000000e+00
Information Ratio,4.662639e+00


# Section 6| FX Carry

- Find the data in the 'final_exam_data.xlsx' on sheet 'fx rates'
- These are quoted in USD per MXN
- Find the interest rates for both countries on sheet 'risk-free rates'
- **6.1** Transform the log FX prices and log interest rates

In [50]:
fx_data = pd.read_excel(final_exam_2021,sheet_name = 'fx rates').set_index("date")
risk_free = pd.read_excel(final_exam_2021, sheet_name = 'risk-free rates').set_index('date')
fx_data = np.log(fx_data)
risk_free = np.log(risk_free+1)

In [51]:
fx_data.mean()

USMX   -2.580061
dtype: float64

In [52]:
np.exp(risk_free.mean())

RF-USD    1.001724
RF-MXN    1.006597
dtype: float64

In [53]:
fx_data.head()

,USMX
date,
1999-01-31,-2.317671
1999-02-28,-2.296215
1999-03-31,-2.253500
1999-04-30,-2.223812
1999-05-31,-2.281617


In [54]:
risk_free.head()

,RF-USD,RF-MXN
date,,
1999-01-31,0.004133,0.029114
1999-02-28,0.004180,0.025790
1999-03-31,0.004158,0.021737
1999-04-30,0.004149,0.018977
1999-05-31,0.004213,0.019198


In [55]:
fx_growth = fx_data.diff().dropna()

log_rf_spread = pd.DataFrame((risk_free['RF-MXN']-risk_free['RF-USD']).shift(1).dropna(), columns = ['Log-Spread'])
fx_growth

,USMX
date,
1999-02-28,0.021456
1999-03-31,0.042715
1999-04-30,0.029687
1999-05-31,-0.057804
1999-06-30,0.036343
...,...
2021-06-30,0.001205
2021-07-31,0.003019
2021-08-31,-0.011423


In [70]:
excess_return = pd.DataFrame(fx_growth['USMX'] + log_rf_spread['Log-Spread'],columns = ['Excess Return'])
excess_return.head()

,Excess Return
date,
1999-02-28,0.046437
1999-03-31,0.064325
1999-04-30,0.047266
1999-05-31,-0.042976
1999-06-30,0.051328


In [57]:
performance_summary(excess_return)

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.5),CVaR (0.5),Max Drawdown,Peak,Bottom,Recovery
Excess Return,0.027273,0.110097,0.24772,-1.129114,4.357954,-0.047702,-0.07903,-0.353741,2008-07-31,2017-01-31,NaT


**6.3**

- Is it better to long or short mxn?
- It is better to go long MXN
  

In [59]:
fx_rf_df = fx_growth.join(log_rf_spread, how = 'inner')
fx_rf_df.mean()

USMX         -0.002583
Log-Spread    0.004856
dtype: float64

- **6.4**
- Assume that these monthly excess log returns are normally distributed. What is the probability that holding MXN over the following 10 years will have a negative cumulative return?

In [72]:
log_RX = excess_return['Excess Return']+risk_free['RF-USD']

In [75]:
mu = log_RX.mean()
sigma = log_RX.std()
periods = 120
norm.cdf(-np.sqrt(periods)*(mu/sigma))

0.08585982261119213

**6.5**

- Based on regresion results, if we observe an increase in the interest rate on MXN  relative to USD, should we expect the USD to get stronger or weaker?

In [80]:
log_rf_spread_1 = -1*log_rf_spread
log_rf_spread_1
mod = LinearRegression().fit(log_rf_spread_1,fx_growth)

FXpredictOLS = pd.DataFrame(
    {'alpha':mod.intercept_[0],
     'beta':mod.coef_[0,0],
     'r-squared':mod.score(log_rf_spread_1,fx_growth)},
    index=['MXN Growth'])

FXpredictOLS

,alpha,beta,r-squared
MXN Growth,-0.007903,-1.095396,0.011123
